In [1]:
from db_utils import *

In [ ]:
import sys

from argparse import Namespace
import dragonfly
from dragonfly import load_config_file
from dragonfly.exd.experiment_caller import CPFunctionCaller
from dragonfly.opt import gp_bandit

In [3]:
#add_component_type(Chemical('Ca(BF4)2','Calcium tetrafluoroborate (hydrate) fisher scientific',213.685,3.09,True))
#remove_electrolyte_by_id(1)
#add_electrolyte({'HC': 32},5,1,1)
#3
#generate_test_electrolytes(1000)
print(get_experiment_electrolytes(['LiF6P', 'CaCl2O8','CaCl2']))

For Electrolyte ID 1, Components are: [1, 2]
Electrolyte ID 1 is valid
For Electrolyte ID 2, Components are: [1, 3]
Electrolyte ID 2 is valid
For Electrolyte ID 3, Components are: [1, 4]
Electrolyte ID 3 is NOT valid
For Electrolyte ID 4, Components are: [1, 5]
Electrolyte ID 4 is NOT valid
For Electrolyte ID 5, Components are: [1, 6]
Electrolyte ID 5 is NOT valid
For Electrolyte ID 6, Components are: [1, 7]
Electrolyte ID 6 is NOT valid
For Electrolyte ID 7, Components are: [1, 8]
Electrolyte ID 7 is NOT valid
For Electrolyte ID 8, Components are: [1, 9]
Electrolyte ID 8 is NOT valid
For Electrolyte ID 9, Components are: [2, 3]
Electrolyte ID 9 is valid
For Electrolyte ID 10, Components are: [2, 4]
Electrolyte ID 10 is NOT valid
For Electrolyte ID 11, Components are: [2, 5]
Electrolyte ID 11 is NOT valid
For Electrolyte ID 12, Components are: [2, 6]
Electrolyte ID 12 is NOT valid
For Electrolyte ID 13, Components are: [2, 7]
Electrolyte ID 13 is NOT valid
For Electrolyte ID 14, Compon

In [ ]:
def get_relev_electrolyte(components: list):
    '''
    takes in list of components, ex: ["formula1", "formula2"]
    returns id of an electrolyte.
    '''
    conn = sqlite3.connect(DB)
    c = conn.cursor()

    #creating long query string with all the requirements for formula
    query_string = "SELECT e.id FROM electrolytes e WHERE e.id IN (SELECT ec.electrolyte_id FROM electrolyte_components ec WHERE "
    query_conditions = []
    query_values = []
    for formula in components:
        chemical = Chemical(formula)
        query_conditions.append("ec.component_id = (SELECT id FROM components WHERE formula = ?)")
        query_values.append(str(chemical))
    query_string += " OR ".join(query_conditions) + ")"
    c.execute(query_string, query_values)
    candidate_ids = [row[0] for row in c.fetchall()]
    conn.commit()
    conn.close()
    
    if(len(candidate_ids) > 1):
        raise ValueError(f"{len(candidate_ids)} total duplicate electrolytes found with components {str(components)}")
    return candidate_ids

In [ ]:
import json

def generate_config(components: dict[
    str, tuple[ #should include chemical formula
    float, float]], #should include lower bound, and upper bound
    min_volume: float,
    max_volume: float,
    min_total_molarity: float,
    max_total_molarity: float
    ):
    

In [5]:
config = load_config_file('./config/config.json')
#optimization method is bayesian optimization
domain, domain_orderings = config.domain, config.domain_orderings

In [6]:
func_caller = CPFunctionCaller(None, domain, domain_orderings=domain_orderings)
opt = gp_bandit.CPGPBandit(func_caller, ask_tell_mode=True)

In [7]:
opt.initialise()

In [6]:
import random
tell = [([random.uniform(.25, 7.25) for _ in range(5)], random.uniform(0, 100)) for _ in range(1)]
print(tell)

[([3.87459089371169, 6.570381775771222, 6.066995164223458, 6.53990002612618, 3.144427432067336], 68.44190725684066)]


In [9]:
opt.tell(tell)

In [17]:
#print(opt.tell(tell))
print(opt.ask())

KeyboardInterrupt: 